In [ ]:
import os
import cv2  # pip install opencv-python
import numpy as np
from tqdm.notebook import tqdm

[Скачать датасет с картинками собак и кошек](https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip)

In [ ]:
REBUILD_DATA = False # set to true to one once, the notebook back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "d:/PetImages/Cat"
    DOGS = "d:/PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0
    
    def convert_img(self, fname):
        img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
        return np.array(img)
    
    def all_pics_paths(self):
        res = []
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
#                         converted_img = self.convert_img(path)
                        res.append((path,self.LABELS[label]))
                    except Exception as e:
                        pass
        np.random.shuffle(res)
        return res

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        m = self.convert_img(path)
                        self.training_data.append([m, np.eye(2)[self.LABELS[label]]]) 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)
        
    def load_training_data(self):
        self.training_data = np.load("training_data.npy",allow_pickle=True)
        self.catcount = 0
        self.dogcount = 0
        for m, label in self.training_data:
            if label[0] == 1:
                self.catcount += 1
            if label[1] == 1:
                self.dogcount +=1
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)
        
    
        
dogsvcats = DogsVSCats()

if REBUILD_DATA:
    dogsvcats.make_training_data()
else:
    dogsvcats.load_training_data()


In [ ]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt


all_pics = dogsvcats.all_pics_paths()

@interact(i=IntSlider(min=0, max=len(all_pics)-1, step=1, value=0))
def print_some(i): 
    fpath, label = all_pics[i]
    img = cv2.imread(fpath)
    converted_img = dogsvcats.convert_img(fpath)
    who = 'ПЁсель' if label else 'КОтель'
    
    plt.subplot(121)
    plt.imshow(img[:,:,::-1])
    plt.subplot(122)
    plt.imshow(converted_img, cmap='gray')
    plt.title(f'На картинке {who}')
    plt.show()

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 conv
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = T.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).
        
    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x   
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)
    
net = Net()
print(net)

In [ ]:
sum([p.numel() for p in net.parameters()])

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

training_data = dogsvcats.training_data
X = T.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = T.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

BATCH_SIZE = 100
EPOCHS = 1

In [ ]:
def train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            net.zero_grad()

            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")
        
def test(net):
    correct = 0
    total = 0
    with T.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i])
            net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
            predicted_class = T.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

In [ ]:
train(net)

In [ ]:
if T.cuda.is_available():
    device = T.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = T.device("cpu")
    print("Running on the CPU")

In [ ]:
netg = Net().to(device)
train_Xg = train_X.to(device)
train_yg = train_y.to(device)

In [ ]:
def train(net, ep):
    BATCH_SIZE = 100
    EPOCHS = ep
    optimizer = optim.Adam(netg.parameters(), lr=0.001)
    loss_function = nn.MSELoss()
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_Xg[i:i+BATCH_SIZE].view(-1,1,50,50)
            batch_y = train_yg[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X, batch_y

            netg.zero_grad()
            outputs = netg(batch_X)

            matches  = [T.argmax(i)==T.argmax(j) for i, j in zip(outputs, batch_y)]
            in_sample_acc = matches.count(True)/len(matches)

            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
        print(loss)
        print("In-sample acc:",round(in_sample_acc, 2))
        
train(netg, 3)

In [ ]:
def batch_test(net):
    BATCH_SIZE = len(test_y)
    correct = 0
    total = 0
    with T.no_grad():
        #np.random.shuffle(test_X)
        #np.random.shuffle(test_y)

        batch_X = test_X[:BATCH_SIZE].view(-1,1,50,50)
        batch_y = test_y[:BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        net.zero_grad()
        outputs = netg(batch_X)

        matches, notmatches  = [], []
        for i, (o, b) in enumerate(zip(outputs, batch_y)):
            if T.argmax(o)!=T.argmax(b):
                notmatches.append(test_X[i])
            else:
                matches.append(test_X[i])
        acc = 1-len(notmatches)/BATCH_SIZE

        print("Test Accuracy:", round(acc, 3))
        return matches, notmatches

rights, wrongs = batch_test(netg)

In [ ]:
@interact(i=IntSlider(min=0, max=len(wrongs)-1, step=1, value=0))
def plot_wrongs(i):
    label=T.argmax(netg(wrongs[i].view(-1,1,50,50).to(device)))
    plt.imshow(wrongs[i], cmap='gray')
    who = 'ПЁсель' if label else 'КОтель'
    plt.title(f'Моя сеть считает, что это {who}')
    plt.show()

In [ ]:
@interact(i=IntSlider(min=0, max=len(rights)-1, step=1, value=0))
def plot_wrongs(i):
    label=T.argmax(netg(rights[i].view(-1,1,50,50).to(device)))
    plt.imshow(rights[i], cmap='gray')
    who = 'ПЁсель' if label else 'КОтель'
    plt.title(f'Моя сеть считает, что это {who}')
    plt.show()